In [4]:
import os 
import sys 
import logging 

logging.basicConfig(level=logging.INFO)

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy
from python_graphql_client import GraphqlClient


from utils import remove_prefix, ddf 

load_dotenv()

True

In [5]:
sg = Subgrounds()
url = os.environ['SUBGRAPH_URL'] 
bs: Subgraph = sg.load_subgraph(url)
client = GraphqlClient(endpoint=url)


In [6]:
# df_ftokens = df_fert_tokens_raw.copy()

# nearest = alt.selection_single(
#     nearest=True, encodings=["x"], on="mouseover", empty="none"
# )
# brush = alt.selection_interval(encodings=['x'])

# base = alt.Chart(df_ftokens)
# x = alt.X("humidity:Q")
# y = alt.Y("fertilizer:Q")

# # Histogram of fertilizer purchased by season 
# histogram = base.mark_bar().encode(
#     x=x, y=y
# )

# # Vertical line to indicate nearest histogram bin 
# vline = base.mark_rule(color="grey").encode(
#     x=x, size=alt.value(1)
# ).transform_filter(
#     nearest
# )

# # Text label to indicate value nearest histogram bin 
# text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
#     text=alt.condition(nearest, y, alt.value(' '))
# )

# # Time axis which drives scale of histogram 
# time_axis = base.mark_bar().encode(
#     x=x, y=y
# ).add_selection(
#     brush 
# ).resolve_scale(
#     'independent'
# ).properties(height=50)

# (
#     alt.layer(
#         histogram.add_selection(nearest), 
#         text, 
#         vline
#     ).transform_filter(
#         brush
#     ).properties(height=250) 
#     & time_axis 
# ) 

### Fertilizer Purchased by Humidity 

In [7]:
seasons = bs.Query.seasons(
    first=10000, where={"season_gte": 6074}, orderBy="season", orderDirection="asc"
)
df_seasons_raw = sg.query_df([
    seasons.season, 
    seasons.timestamp, 
], pagination_strategy=ShallowStrategy)
df_seasons_raw = remove_prefix(df_seasons_raw, 'seasons_')
print(len(df_seasons_raw))
df_seasons_raw.head()

INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  xea11476953cb434e: seasons(first: $first0, skip: $skip0, orderBy: season, orderDirection: asc, where: {season_gte: 6074}) {
    season
    timestamp
    id
  }
}


299


,season,timestamp
0,6074,1650196810
1,6075,1659805214
2,6076,1659808931
3,6077,1659812464
4,6078,1659816002


In [8]:
# bs.FertilizerToken.sprouts = (
#     bs.FertilizerToken.supply * (bs.FertilizerToken.humidity + 100) / 100
# ) 
bs.FertilizerToken.fert = bs.FertilizerToken.supply 
bs.FertilizerToken.endBpf = SyntheticField(
  lambda _id: float(_id) / 1e6, 
  SyntheticField.FLOAT,
  bs.FertilizerToken.id, 
)
fert_tokens = bs.Query.fertilizerTokens(
    first=10000, 
    orderBy="humidity", 
    orderDirection="desc"
)
df_fert_tokens_raw = sg.query_df(
    [
        fert_tokens.season, 
        fert_tokens.fert, 
        fert_tokens.humidity, 
        fert_tokens.startBpf, 
        fert_tokens.endBpf, 
    ],
    pagination_strategy=ShallowStrategy
)
df_fert_tokens_raw = remove_prefix(df_fert_tokens_raw, "fertilizerTokens_")
df_fert_tokens_raw['startBpf'] /= 1e6
print(len(df_fert_tokens_raw))
df_fert_tokens_raw.head(10)


INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  x9a5b0bd117500c08: fertilizerTokens(first: $first0, skip: $skip0, orderBy: humidity, orderDirection: desc, where: {}) {
    season
    supply
    humidity
    startBpf
    id
  }
}


87


,season,fert,humidity,startBpf,endBpf
0,6074,14364122,500.0,0.000000,6.000000
1,6074,70842,250.0,0.000000,3.500000
2,6075,3032,249.5,0.000000,3.495000
3,6076,31160,249.0,0.000157,3.490157
4,6077,11003,248.5,0.000472,3.485472
5,6078,4383,248.0,0.000951,3.480951
6,6079,15753,247.5,0.001597,3.476597
7,6080,15388,247.0,0.002520,3.472520
8,6081,100,246.5,0.003691,3.468691
9,6082,3095,246.0,0.005087,3.465087


In [9]:
rewards = bs.Query.rewards(orderBy="blockNumber", orderDirection="asc", first=10000)

df_rewards_raw = sg.query_df([
    rewards.season, 
    rewards.toFertilizer, 
], pagination_strategy=ShallowStrategy)
df_rewards_raw = remove_prefix(df_rewards_raw, 'rewards_')
df_rewards_raw['toFertilizer'] = df_rewards_raw['toFertilizer'].astype(float) / 1e6 
df_rewards_raw.head()

INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  xaa8e723fbd9668e6: rewards(first: $first0, skip: $skip0, orderBy: blockNumber, orderDirection: asc, where: {}) {
    season
    toFertilizer
    id
    blockNumber
  }
}


,season,toFertilizer
0,6076,2266.788451
1,6077,4557.830445
2,6078,6936.066574
3,6079,9357.109094
4,6080,13383.907966


### Experiment  

In [10]:
df_seasons_raw.head() 

,season,timestamp
0,6074,1650196810
1,6075,1659805214
2,6076,1659808931
3,6077,1659812464
4,6078,1659816002


In [11]:
df_fert_tokens_raw.head()

,season,fert,humidity,startBpf,endBpf
0,6074,14364122,500.0,0.000000,6.000000
1,6074,70842,250.0,0.000000,3.500000
2,6075,3032,249.5,0.000000,3.495000
3,6076,31160,249.0,0.000157,3.490157
4,6077,11003,248.5,0.000472,3.485472


In [12]:

df = df_seasons_raw.merge(
    df_rewards_raw, how="left", on="season"
).merge(
    df_fert_tokens_raw, how="left", on="season"
)
df = df.rename(columns={"toFertilizer": "to_fertilizer"})
df['timestamp'] = pd.to_datetime(df.timestamp, unit="s")

df['to_fertilizer'] = df.to_fertilizer.fillna(0)
df['fertilized_beans'] = df.to_fertilizer.cumsum()
df['unfertilized_beans'] = ((df.fert * df.endBpf).cumsum() - df.fertilized_beans).ffill()
df['sprouts_rinsable'] = df.fertilized_beans
df['sprouts'] = df.unfertilized_beans
df['rinsable_percent'] = df.fertilized_beans / (df.fertilized_beans + df.unfertilized_beans).ffill()

df['fertilizer_total'] = 77000000
df['fertilizer_active'] = df.fert.cumsum().ffill()
df['fertilizer_available'] = (df.fertilizer_total - df.fertilizer_active).ffill()

# ddf(df)
df = df.sort_values("timestamp")
df = df.loc[df.season >= 6075]
df.head(10)

,season,timestamp,to_fertilizer,fert,humidity,startBpf,endBpf,fertilized_beans,unfertilized_beans,sprouts_rinsable,sprouts,rinsable_percent,fertilizer_total,fertilizer_active,fertilizer_available
2,6075,2022-08-06 17:00:14,0.000000,3032.0,249.5,0.000000,3.495000,0.000000,8.644328e+07,0.000000,8.644328e+07,0.000000,77000000,14437996.0,62562004.0
3,6076,2022-08-06 18:02:11,2266.788451,31160.0,249.0,0.000157,3.490157,2266.788451,8.654976e+07,2266.788451,8.654976e+07,0.000026,77000000,14469156.0,62530844.0
4,6077,2022-08-06 19:01:04,4557.830445,11003.0,248.5,0.000472,3.485472,6824.618896,8.658356e+07,6824.618896,8.658356e+07,0.000079,77000000,14480159.0,62519841.0
5,6078,2022-08-06 20:00:02,6936.066574,4383.0,248.0,0.000951,3.480951,13760.685470,8.659188e+07,13760.685470,8.659188e+07,0.000159,77000000,14484542.0,62515458.0
6,6079,2022-08-06 21:00:00,9357.109094,15753.0,247.5,0.001597,3.476597,23117.794564,8.663729e+07,23117.794564,8.663729e+07,0.000267,77000000,14500295.0,62499705.0
7,6080,2022-08-06 22:00:04,13383.907966,15388.0,247.0,0.002520,3.472520,36501.702530,8.667734e+07,36501.702530,8.667734e+07,0.000421,77000000,14515683.0,62484317.0
8,6081,2022-08-06 23:00:33,16998.036930,100.0,246.5,0.003691,3.468691,53499.739460,8.666069e+07,53499.739460,8.666069e+07,0.000617,77000000,14515783.0,62484217.0
9,6082,2022-08-07 00:00:03,20264.238280,3095.0,246.0,0.005087,3.465087,73763.977740,8.665115e+07,73763.977740,8.665115e+07,0.000851,77000000,14518878.0,62481122.0
10,6083,2022-08-07 01:00:08,23332.073175,2987.0,245.5,0.006694,3.461694,97096.050915,8.663815e+07,97096.050915,8.663815e+07,0.001119,77000000,14521865.0,62478135.0
11,6084,2022-08-07 02:00:06,26676.936044,59400.0,245.0,0.008531,3.458531,123772.986959,8.681691e+07,123772.986959,8.681691e+07,0.001424,77000000,14581265.0,62418735.0


In [13]:
def visualize_recap_paid_percent(df: pd.DataFrame): 
    """"""
    df = df.iloc[1:,]
    # fertilizer breakdown 
    series_fert = df.iloc[-1,][['fertilizer_available', 'fertilizer_active']]
    fert_total = series_fert['fertilizer_available'] + series_fert['fertilizer_active']
    df_fert = pd.DataFrame(
        {
            'category': series_fert.index, 'value': series_fert.values, 
            'label': [
                f"{c}: {int(v):,} \n({v / fert_total:.1%})" 
                for c, v in zip(series_fert.index, series_fert.values)
            ]
        }
    )
    fertililzer_breakdown = alt.Chart(
        df_fert, title="Fertilizer Breakdown"
    ).encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color("category:N", legend=alt.Legend()),
        tooltip="label:N"
    ).mark_arc(outerRadius=120)
    # sprouts breakdown 
    df_sprouts = df.copy()
    label_vars = ['sprouts rinsable', 'sprouts ', 'rinsable percent']
    id_vars = ['timestamp'] + label_vars
    value_vars = ['sprouts_rinsable', 'sprouts', 'rinsable_percent']
    df_sprouts['sprouts rinsable'] = df_sprouts['sprouts_rinsable']
    df_sprouts['sprouts '] = df_sprouts['sprouts']
    df_sprouts['rinsable percent'] = df_sprouts['rinsable_percent']
    df_sprouts = df_sprouts.melt(
        id_vars=id_vars, value_vars=value_vars
    ).sort_values("timestamp")
    sprouts_base = alt.Chart(
        df_sprouts[['timestamp', 'value', 'variable'] + label_vars], 
        title="Sprouts Breakdown"
    ).encode(
        x=alt.X("yearmonthdatehours(timestamp):T", axis=alt.Axis(tickCount=10, title="Date")), 
        y=alt.Y("value:Q", axis=alt.Axis(title="Sprouts")), 
        color=alt.Color("variable:N", legend=alt.Legend()),
    )
    sprouts_area = sprouts_base.transform_filter(
        datum.variable != 'rinsable_percent'
    ).mark_area(point="transparent").encode(
        tooltip=[
            alt.Tooltip(f'{e}:Q', format=",d" if e != 'rinsable percent' else ".2%") 
            for e in label_vars
        ]
    )
    sprouts_line = sprouts_base.transform_filter(
        datum.variable == 'rinsable_percent'
    ).mark_line().encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Rinsable Percent", format=".2%")), 
    )
    sprouts_breakdown = alt.layer(sprouts_area, sprouts_line).resolve_scale(
        y="independent", 
    ).resolve_axis(
        y="independent"
    ).properties(width=400)
    
    return alt.hconcat(
        sprouts_breakdown, 
        fertililzer_breakdown
    ).resolve_legend(
        color="independent", 
    ).resolve_scale(
        y="independent",
        color="independent",
    ).resolve_axis(
        y="independent"
    )
 
visualize_recap_paid_percent(df)

alt.HConcatChart(...)